In [1]:
from eda_imports import *

import matplotlib.patches as patches

### Load GTF

In [2]:
%time df_gtf = pd.read_pickle('/projects/btl2/zxue/gtf2csv/data/Homo_sapiens.GRCh38.92.pkl')

CPU times: user 1.39 s, sys: 776 ms, total: 2.17 s
Wall time: 2.18 s


In [3]:
ndf_gtf = df_gtf.query('feature != "gene"').query('feature != "transcript"').drop(['source', 'gene_source'], axis=1)

# See how to common it is to have everything the same but 3' UTR for two transcripts

In [4]:
ndf_gtf.head(2)

,seqname,feature,start,end,score,strand,frame,CCDS,basic,ccds_id,cds_end_NF,cds_start_NF,exon_id,exon_number,exon_version,gene_biotype,gene_id,gene_name,gene_version,mRNA_end_NF,mRNA_start_NF,protein_id,protein_version,seleno,transcript_biotype,transcript_id,transcript_name,transcript_source,transcript_support_level,transcript_version
2,1,exon,11869,12227,.,+,.,NaN,1.0,NaN,NaN,NaN,ENSE00002234944,1.0,1.0,transcribed_unprocessed_pseudogene,ENSG00000223972,DDX11L1,5,NaN,NaN,NaN,NaN,NaN,processed_transcript,ENST00000456328,DDX11L1-202,havana,1,2.0
3,1,exon,12613,12721,.,+,.,NaN,1.0,NaN,NaN,NaN,ENSE00003582793,2.0,1.0,transcribed_unprocessed_pseudogene,ENSG00000223972,DDX11L1,5,NaN,NaN,NaN,NaN,NaN,processed_transcript,ENST00000456328,DDX11L1-202,havana,1,2.0


In [5]:
ndf_gtf.feature.value_counts()

exon               1237303
CDS                734649 
five_prime_utr     147919 
three_prime_utr    144546 
start_codon        85453  
stop_codon         77440  
Selenocysteine     119    
Name: feature, dtype: int64

In [6]:
assert df_gtf.gene_id.unique().shape[0] == df_gtf[['gene_id', 'gene_biotype']].drop_duplicates().shape[0]

In [9]:
def count_transcripts_with_identical_cds(grp):
    """grp: grouped by gene_id"""
    cds_list = []
    for k, g in grp.query('transcript_biotype == "protein_coding"').groupby('transcript_id'):
        cds_start_end_pairs = g.query('feature == "CDS"')[['start', 'end']].sort_values(['start', 'end']).values.tolist()
        cds_start_end_pairs_str = str(cds_start_end_pairs)
        cds_list.append(cds_start_end_pairs_str)
    return pd.Series(cds_list).value_counts().max()

In [10]:
keys, args = [], []
grp_by_cols = ['gene_name', 'gene_id', 'gene_biotype']
for k, grp in tqdm(ndf_gtf.groupby(grp_by_cols)):
    keys.append(k)
    args.append(grp)


100%|██████████| 58395/58395 [00:14<00:00, 4033.47it/s]

In [11]:
%%time
with multiprocessing.Pool(50) as p:
    res = p.map(count_transcripts_with_identical_cds, args)

CPU times: user 21.6 s, sys: 6.14 s, total: 27.7 s
Wall time: 39.7 s


In [12]:
# number of genes with transcripts that share identical cds regions
df_count = pd.DataFrame(keys, columns=grp_by_cols)

In [13]:
df_count['num'] = res

In [14]:
df_count.head()

,gene_name,gene_id,gene_biotype,num
0,A1BG,ENSG00000121410,protein_coding,1.0
1,A1BG-AS1,ENSG00000268895,antisense,NaN
2,A1CF,ENSG00000148584,protein_coding,3.0
3,A2M,ENSG00000175899,protein_coding,1.0
4,A2M-AS1,ENSG00000245105,antisense,NaN


In [15]:
df_count.query('gene_biotype == "protein_coding"').query('num > 1').shape

(5584, 4)

In [16]:
5584 / 19912

0.2804339092004821

So 28% of the genes have protein_coding transcripts sharing identical CDS

In [17]:
ndf_count = df_count.query('gene_biotype == "protein_coding"')

In [19]:
ndf_count.query('gene_name == "SPG7"')

,gene_name,gene_id,gene_biotype,num
52971,SPG7,ENSG00000197912,protein_coding,1.0


In [54]:
ndf_count.sort_values('num', ascending=False).head(10)

,gene_name,gene_id,gene_biotype,num
24023,CTNNB1,ENSG00000168036,protein_coding,24.0
51819,SLC6A1,ENSG00000157103,protein_coding,19.0
34586,MAPK10,ENSG00000109339,protein_coding,18.0
37349,MRPL55,ENSG00000162910,protein_coding,16.0
20691,BDNF,ENSG00000176697,protein_coding,14.0
18375,ANAPC11,ENSG00000141552,protein_coding,13.0
57469,ZBTB7C,ENSG00000184828,protein_coding,12.0
27941,GJB6,ENSG00000121742,protein_coding,11.0
26144,ESRRG,ENSG00000196482,protein_coding,11.0
21641,C6orf48,ENSG00000204387,protein_coding,10.0


# Debug

In [49]:
lele = ndf_gtf.query('gene_name == "SLC6A1"')

In [50]:
cds_list = []
# for k, g in lele.query('transcript_biotype == "protein_coding"').groupby('transcript_id'):
for k, g in lele.groupby('transcript_id'):
    assert g.transcript_id.unique().shape[0] == 1
    assert g.transcript_biotype.unique().shape[0] == 1
    transcript_id = g.transcript_id.unique()[0]
    transcript_biotype = g.transcript_biotype.unique()[0]
    
    cds_start_end_pairs = g.query('feature == "CDS"')[['start', 'end']].sort_values(['start', 'end']).values.tolist()
    cds_start_end_pairs_str = str(cds_start_end_pairs)
    cds_list.append((transcript_id, transcript_biotype, hash(cds_start_end_pairs_str)))
# pd.Series(cds_list).value_counts().max()

In [51]:
df_cds_list = pd.DataFrame(cds_list, columns=['transcript_id', 'transcript_biotype', 'cds_hash'])

In [52]:
df_cds_list.transcript_id.unique().shape

(41,)

In [53]:
df_cds_list.groupby('cds_hash').apply(lambda g: (g.transcript_biotype.unique(), g.shape[0], g.transcript_id.unique()))

cds_hash
-7669697834645599824    ([protein_coding], 19, [ENST00000287766, ENST00000642201, ENST00000642515, ENST00000642639, ENST00000642735, ENST00000642767, ENST00000642820, ENST00000643396, ENST00000643498, ENST00000645029, ENST00000645054, ENST00000645592, ENST00000645974, ENST00000646022, ENST00000646060, ENST00000646570, ENST00000646702, ENST00000646924, ENST00000647194])
-6821238873617149842    ([protein_coding], 1, [ENST00000646487])                                                                                                                                                                                                                                                                                                                   
-5675111249409555424    ([protein_coding], 1, [ENST00000647384])                                                                                                                                                                                       

In [22]:
lele = ndf_gtf.query('gene_name == "SPG7"')

In [33]:
cds_list = []
# for k, g in lele.query('transcript_biotype == "protein_coding"').groupby('transcript_id'):
for k, g in lele.groupby('transcript_id'):
    assert g.transcript_id.unique().shape[0] == 1
    assert g.transcript_biotype.unique().shape[0] == 1
    transcript_id = g.transcript_id.unique()[0]
    transcript_biotype = g.transcript_biotype.unique()[0]
    
    cds_start_end_pairs = g.query('feature == "CDS"')[['start', 'end']].sort_values(['start', 'end']).values.tolist()
    cds_start_end_pairs_str = str(cds_start_end_pairs)
    cds_list.append((transcript_id, transcript_biotype, hash(cds_start_end_pairs_str)))
# pd.Series(cds_list).value_counts().max()

In [36]:
df_cds_list = pd.DataFrame(cds_list, columns=['transcript_id', 'transcript_biotype', 'cds_hash'])

In [39]:
df_cds_list.transcript_id.unique().shape

(96,)

In [42]:
df_cds_list.groupby('cds_hash').apply(lambda g: (g.transcript_biotype.unique(), g.shape[0]))

cds_hash
-8977407504005217083    ([nonsense_mediated_decay], 1)                
-8496234967062161228    ([nonsense_mediated_decay], 1)                
-8403022975375736636    ([nonsense_mediated_decay], 1)                
-8239834149324152343    ([nonsense_mediated_decay], 1)                
-7003683457219581767    ([nonsense_mediated_decay], 1)                
-6967304914575017851    ([nonsense_mediated_decay], 1)                
-6870317669951290742    ([protein_coding], 1)                         
-6779581224490335370    ([protein_coding], 1)                         
-6509607191405754552    ([protein_coding], 1)                         
-6164088867915897106    ([nonsense_mediated_decay], 1)                
-5675297078241503365    ([protein_coding], 1)                         
-5597205556425114475    ([nonsense_mediated_decay], 1)                
-5441304313678483069    ([nonsense_mediated_decay], 1)                
-5369735789868731396    ([nonsense_mediated_decay], 1)              